In [0]:
%sql

ALTER TABLE 5_projects.dac023_pacs.request_20250915
ADD COLUMNS (PERSON_ID INT)


In [0]:
%sql
WITH nhs_nbr AS (
    SELECT
        MAX(PERSON_ID) AS PERSON_ID,
        ALIAS AS NHS_Number
    FROM 4_prod.rde.patient_nhs
    GROUP BY ALIAS
)
MERGE INTO 5_projects.dac023_pacs.request_20250915 AS dest
USING nhs_nbr
ON CAST(dest.NHS_Number AS STRING)= nhs_nbr.NHS_Number
WHEN MATCHED THEN
    UPDATE SET dest.PERSON_ID = nhs_nbr.PERSON_ID

In [0]:
%sql

CREATE TABLE 5_projects.dac023_pacs.request_20250915_accession_nbr (
    person_id INT,
    accession_nbr STRING
)

In [0]:
%sql
TRUNCATE TABLE 5_projects.dac023_pacs.request_20250915_accession_nbr;

In [0]:
%sql
INSERT INTO 5_projects.dac023_pacs.request_20250915_accession_nbr (accession_nbr, person_id)
SELECT
    DISTINCT m.AccessionNbr,
    r.PERSON_ID
FROM 4_prod.pacs.mill_pacs_data_expanded m
INNER JOIN 5_projects.dac023_pacs.request_20250915 r
    ON m.MillPersonId = r.PERSON_ID